In [1]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from faker import Faker
import random
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam


2024-05-16 17:40:24.309179: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 17:40:24.309325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 17:40:24.444493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Set the number of features in your dataset
number_of_features = 5

fake = Faker()

# Define the number of samples you want to generate
number_of_samples = 1000

# Define the structure of your synthetic dataset
data = {
    "transaction_id": [fake.uuid4() for _ in range(number_of_samples)],
    "timestamp": [fake.date_time_this_decade() for _ in range(number_of_samples)],
    "amount": [random.uniform(10.0, 1000.0) for _ in range(number_of_samples)],
    "merchant": [fake.company() for _ in range(number_of_samples)],
    "category": [
        fake.random_element(
            elements=(
                "Grocery",
                "Electronics",
                "Clothing",
                "Entertainment",
                "Utilities",
            )
        )
        for _ in range(number_of_samples)
    ],
    "customer_id": [
        fake.random_int(min=1000, max=9999) for _ in range(number_of_samples)
    ],
    "fraudulent": [
        fake.boolean(chance_of_getting_true=5) for _ in range(number_of_samples)
    ],  # 5% chance of a transaction being fraudulent
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the dataset to a CSV file
df.to_csv("synthetic_fraud_dataset.csv", index=False)


In [4]:
# Define the generator model
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LeakyReLU, BatchNormalization, Reshape
import numpy as np


def build_generator():
    noise_shape = (100,)  # Dimension of the random noise

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(2, activation="tanh"))  # Assuming 2 output features for simplicity

    noise = Input(shape=noise_shape)
    synthetic_data = model(noise)

    return Model(noise, synthetic_data)

    # Define the discriminator model


from keras.layers import Flatten, Dropout
from keras.optimizers import Adam


def build_discriminator():
    model = Sequential()

    model.add(Dense(512, input_shape=(2,)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Dense(1, activation="sigmoid"))

    data = Input(shape=(2,))
    validity = model(data)

    return Model(data, validity)

    # Build and compile the discriminator


discriminator = build_discriminator()
discriminator.compile(
    loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5), metrics=["accuracy"]
)

# Build the generator
generator = build_generator()

# Define the combined model (generator + discriminator)
z = Input(shape=(100,))
synthetic_data = generator(z)
discriminator.trainable = False
validity = discriminator(synthetic_data)

gan = Model(z, validity)
gan.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [5]:
# Training the GAN
def train_gan(generator, discriminator, gan, epochs, batch_size, save_interval):
    for epoch in range(epochs):
        # Train the discriminator
        real_transactions = np.random.normal(
            0, 1, (batch_size, 2)
        )  # Assuming normal distribution for real data
        noise = np.random.normal(0, 1, (batch_size, 100))
        synthetic_transactions = generator.predict(noise)

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        d_loss_real = discriminator.train_on_batch(real_transactions, valid)
        d_loss_fake = discriminator.train_on_batch(synthetic_transactions, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, valid)

        # If at save interval, print the progress
        if epoch % save_interval == 0:
            print(
                f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}%] [G loss: {g_loss}]"
            )
            # Save synthetic transactions
            synthetic_df = pd.DataFrame(
                synthetic_transactions, columns=["feature1", "feature2"]
            )
            synthetic_df.to_csv(
                f"synthetic_transactions_epoch_{epoch}.csv", index=False
            )


# Set parameters
epochs = 10000
batch_size = 32
save_interval = 200

# Train the GAN
train_gan(generator, discriminator, gan, epochs, batch_size, save_interval)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 933ms/step


I0000 00:00:1715881236.479846      72 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:77: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.6889902353286743, acc.: 38.28125%] [G loss: 0.6845062971115112]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━

In [ ]:
# Load the synthetic dataset
df = pd.read_csv("synthetic_fraud_dataset.csv/kaggle/working/synthetic_fraud_dataset.csv")

# Inspect the data
print(df.head())
print(df.describe())
print(df.info())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the dataset into features and labels
X = df.drop(columns=["fraudulent"])
y = df["fraudulent"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Evaluate the model
print(classification_report(y_test, y_pred))
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_proba)}")
